In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [31]:
data = pd.read_csv('Data_folder/robo_coord_angle_dataset.csv')

In [32]:
data.isnull().any()

Unnamed: 0    False
x_coord       False
y_coord       False
z_coord       False
alpha         False
beta          False
gamma         False
dtype: bool

In [33]:
data.head(10)

,Unnamed: 0,x_coord,y_coord,z_coord,alpha,beta,gamma
0,0,12.524,-50.233,1432.786,104,-38,-90
1,1,7.796,-31.269,1432.445,104,-38,-88
2,2,3.074,-12.329,1431.422,104,-38,-86
3,3,-1.637,6.565,1429.718,104,-38,-84
4,4,-6.330,25.389,1427.336,104,-38,-82
5,5,-11.001,44.122,1424.278,104,-38,-80
6,6,-15.643,62.739,1420.549,104,-38,-78
7,7,-20.250,81.219,1416.152,104,-38,-76
8,8,-24.818,99.539,1411.093,104,-38,-74
9,9,-29.340,117.677,1405.378,104,-38,-72


In [34]:
X_df = data.iloc[:, 1:4]
Y_df = data.iloc[:, 4:]
X = X_df.values
Y = Y_df.values

In [38]:
#splitting the dataset into train, valid, test in the ratio  of 6:2:2
Xtrainval, Xtest, Ytrainval, Ytest = train_test_split(X, Y, test_size=0.2, random_state = 42)
Xtrain, Xval, Ytrain, Yval = train_test_split(Xtrainval, Ytrainval, test_size=0.16, random_state = 42)

In [39]:
#scaling (standard) X data as per Xtrain
scaler = StandardScaler()
Xtrain = scaler.fit_transform(Xtrain)
Xval = scaler.transform(Xval)
Xtest = scaler.transform(Xtest)